# Reverse engineer molecules

#### Apr 2024 version

## Enumerate signatures and molecules from Morgan fingerprints

In [1]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"

# packages
import numpy as np
import os

import pandas as pd
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

os.chdir(path_directory)
os.environ["RD_CANON"] = "True"

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import Chem
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

# Reverse engineer molecules from Morgan
from signature.enumerate_signature import enumerate_molecule_from_signature, enumerate_signature_from_morgan
from signature.enumerate_utils import reduced_fingerprint
from signature.Signature import MoleculeSignature
from signature.signature_alphabet import (
    load_alphabet,
    morgan_vector_from_signature,
    SignatureAlphabet,
    signature_from_smiles,
)
from signature.utils import read_csv

Canonicalization of SMARTS is enabled.


In [2]:
# smarts or smiles
use_smarts = False

In [3]:
# paths
file_smiles = "./datasets/MetaNetX_weight_500_radius_3_size_1000"
#file_smiles = "./datasets/emolecule_weight_500_radius_2_size_1000"
#file_smiles = "./datasets/emolecule_weight_500"
#file_smiles = "./datasets/dataset.test.small"

if use_smarts:
    file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_nBits_2048_smarts"
else:
    file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048"

#file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_boundary_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_full_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048"
file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_boundary_nBits_2048_smarts"

# Load Smiles file
H, D = read_csv(file_smiles)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 0])))
print(f"Number of smiles: {len(Smiles)}")

# Get alphabet
Alphabet = load_alphabet(file_alphabet)
Alphabet.print_out()

['SMILES', 'SIG', 'SIG-NBIT'] 1000
Number of smiles: 1000
filename: ./datasets/MetaNetX_weight_500_Alphabet_radius_2_boundary_nBits_2048_smarts.npz
radius: 2
nBits: 2048
splitcomponent: False
isomericSmiles: False
formalCharge: True
atomMapping: False
kekuleSmiles: False
allHsExplicit: False
maxvalence: 4
alphabet length: 183357


In [4]:
# fonctions test
def test_sol_ECFP(smis):
    fpgen = AllChem.GetMorganGenerator(radius=Alphabet.radius, fpSize=2048)
    ecfp_list = []
    for smi in smis:
        mol = Chem.MolFromSmiles(smi)
        morgan = fpgen.GetCountFingerprint(mol).ToList()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_ECFP_reduced(smis):
    ecfp_list = []
    for smi in smis:
        morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False).tolist()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_sig(smis):
    sig_list = []
    for smi in smis:
        mol = Chem.MolFromSmiles(smi)
        ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True)
        sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
        sig_list.append(sig)
    return len(set(tuple(i) for i in sig_list)) == 1

- ECFP reduced => Signature

In [5]:
list_i = []
list_smi = []
list_nsig = []
list_foundsig = []
list_ct = []
list_ct_solve = []
list_timeout = []

In [6]:
Alphabet.nBits = 2048
max_nbr_partition = int(1e5)

for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    st = time.time()

    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ft = time.time() - st

    print(f"{i} | {smi} | {len(Ssig)} | {int(foundsig)} | {ft:.4f} | {ct_solve:.4f}| {bool_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_nsig.append(len(Ssig))
    list_foundsig.append(foundsig)
    list_ct.append(ft)
    list_ct_solve.append(ct_solve)
    list_timeout.append(bool_timeout)

0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 1 | 1 | 2.0590 | 0.0550| False
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 1 | 1 | 1.7910 | 0.0020| False
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 1 | 1 | 2.0781 | 0.0031| False
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C | 1 | 1 | 2.6834 | 0.0040| False
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N | 1 | 1 | 1.7820 | 0.0000| False


In [7]:
100 * sum(list_foundsig) / len(list_foundsig)

100.0

In [8]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "Nsig": list_nsig,
        "FoundSig": list_foundsig,
        "CT enum ecfp_red to sig": list_ct,
        "CT solve by partitions": list_ct_solve,
        "Partitions timeout": list_timeout,
    }
)

df

# df.to_excel("RevSig_ecfp_to_sig", index=False)

,ID,smi,Nsig,FoundSig,CT enum ecfp_red to sig,CT solve by partitions,Partitions timeout
0,0,CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O,1,True,2.058999,0.055003,False
1,1,OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O,1,True,1.790995,0.002000,False
2,2,O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O,1,True,2.078088,0.003088,False
3,3,CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...,1,True,2.683398,0.003995,False
4,4,COC(=O)c1sc(=S)n(-c2ccccc2)c1N,1,True,1.782038,0.000000,False


- Signature => Smiles

In [9]:
list_i = []
list_smi = []
list_wt = []
list_nmol = []
list_foundmol = []
list_ct_mol = []
list_recursion_timeout = []

In [10]:
Alphabet.nBits = 0
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout")
for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))
    
    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)

    st = time.time()
    smol, recursion_timeout = enumerate_molecule_from_signature(
        sig,
        Alphabet,
        smi,
        use_smarts=use_smarts,
        max_nbr_recursion=max_nbr_recursion,
        max_nbr_solution=max_nbr_solution,
        repeat=repeat,
        verbose=False,
    )

    ft = time.time() - st
    foundmol = smi in smol
    if foundmol == False:
        print(smol)

    print(f"{i} | {smi} | {wt:.1f} | {len(smol)} | {int(foundmol)} | {ft:.4f} | {recursion_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nmol.append(len(smol))
    list_foundmol.append(int(foundmol))
    list_ct_mol.append(ft)
    list_recursion_timeout.append(recursion_timeout)

ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 286.1 | 1 | 1 | 0.0220 | False
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 346.0 | 1 | 1 | 0.0259 | False
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 322.1 | 1 | 1 | 0.1180 | False
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C | 420.3 | 2 | 1 | 0.1149 | False
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N | 266.0 | 1 | 1 | 0.0219 | False


In [11]:
100 * sum(list_foundmol) / len(list_foundmol)

100.0

In [12]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "weight": list_wt,
        "Nmol": list_nmol,
        "FoundMol": list_foundmol,
        "CT enum sig to mol": list_ct_mol,
        "Recursion timeout": list_recursion_timeout,
    }
)

df

# df.to_excel("RevSig_sig_to_mol", index=False)

,ID,smi,weight,Nmol,FoundMol,CT enum sig to mol,Recursion timeout
0,0,CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O,286.116486,1,1,0.022001,False
1,1,OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O,345.969705,1,1,0.025928,False
2,2,O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O,322.131742,1,1,0.118003,False
3,3,CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...,420.272318,2,1,0.114895,False
4,4,COC(=O)c1sc(=S)n(-c2ccccc2)c1N,266.018370,1,1,0.021920,False


- ECFP reduced => Signature => Smiles

In [13]:
list_i = []
list_smi = []
list_wt = []
list_nsig = []
list_nsigtrue = []
list_foundsig = []
list_nmol = []
list_foundmol = []
list_ct_sig = []
list_ct_mol = []
list_ct_all = []
list_ct_solve = []

In [14]:
max_nbr_partition = int(1e5)
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time")
for i in range(5):
    # preparation
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    # ecfp_red => signature(s)
    st_1 = time.time()
    Alphabet.nBits = 2048
    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ct_sig = time.time() - st_1
    print(f"...enumeratesignature: nbr signatures {len(Ssig)} CPU-time {ct_sig:.4f}")

    # signature(s) => molecule(s)
    st_2 = time.time()
    Smol, Nsig = set(), 0
    Alphabet.nBits = 0
    for j in range(len(Ssig)):
        smol, recursion_timeout = enumerate_molecule_from_signature(
            Ssig[j],
            Alphabet,
            smi,
            use_smarts=use_smarts,
            max_nbr_recursion=max_nbr_recursion,
            max_nbr_solution=max_nbr_solution,
            repeat=repeat,
            verbose=False,
        )
        if len(smol):
            Nsig += 1
            print(f"...enumeratemolecule:  signature {j} nbr molecules {len(smol)}")
        Smol = Smol | set(smol)
    ct_mol = time.time() - st_2

    ct_total = time.time() - st_1
                
    Smolfinal = []
    for smi2 in Smol:
        if test_sol_ECFP_reduced([smi, smi2]):
            Smolfinal.append(smi2)
    if len(Smol) != len(Smolfinal):
        print("Smol", len(Smol), "Smolfinal", len(Smolfinal))

    foundmol = smi in Smolfinal
    if foundmol == False:
        print(Smolfinal)
        
    print(
        f"{i} | {smi} | {wt:.1f} | {len(Ssig)} | {Nsig} | {int(foundsig)} | {len(Smolfinal)} | {int(foundmol)} | {ct_total:.4f} | {ct_solve:.4f}"
    )
    
    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nsig.append(len(Ssig))
    list_nsigtrue.append(Nsig)
    list_foundsig.append(int(foundsig))
    list_nmol.append(len(Smolfinal))
    list_foundmol.append(int(foundmol))
    list_ct_sig.append(ct_sig)
    list_ct_mol.append(ct_mol)
    list_ct_all.append(ct_total)
    list_ct_solve.append(ct_solve)

ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
...enumeratesignature: nbr signatures 1 CPU-time 2.0400
...enumeratemolecule:  signature 0 nbr molecules 1
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 286.1 | 1 | 1 | 1 | 1 | 1 | 2.0609 | 0.0030
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
...enumeratesignature: nbr signatures 1 CPU-time 1.7891
...enumeratemolecule:  signature 0 nbr molecules 1
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 346.0 | 1 | 1 | 1 | 1 | 1 | 1.8161 | 0.0020
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
...enumeratesignature: nbr signatures 1 CPU-time 2.2446
...enumeratemolecule:  signature 0 nbr molecules 1
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 322.1 | 1 | 1 | 1 | 1 | 1 | 2.3616 | 0.0031
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
...enumeratesignature: nbr signatures 1 CPU-time 2.6870
...enumeratemolecule:  signature 0 nbr molecules 2
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(

In [15]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "wt": list_wt,
        "Nsig": list_nsig,
        "NsigTrue": list_nsigtrue,
        "FoundSig": list_foundsig,
        "Nmol": list_nmol,
        "Foundmol": list_foundmol,
        "CT ecfp_sig": list_ct_sig,
        "CT sig_mol": list_ct_mol,
        "CT ecfp_mol": list_ct_all,
        "CT solve_partitions": list_ct_solve,
    }
)

df

# df.to_excel("RevSig_ecfp_to_mol.xlsx", index=False)

,ID,smi,wt,Nsig,NsigTrue,FoundSig,Nmol,Foundmol,CT ecfp_sig,CT sig_mol,CT ecfp_mol,CT solve_partitions
0,0,CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O,286.116486,1,1,1,1,1,2.039998,0.020919,2.060917,0.003011
1,1,OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O,345.969705,1,1,1,1,1,1.789078,0.026085,1.816086,0.002025
2,2,O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O,322.131742,1,1,1,1,1,2.244576,0.117011,2.361587,0.003089
3,3,CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...,420.272318,1,1,1,2,1,2.687001,0.112999,2.800000,0.003000
4,4,COC(=O)c1sc(=S)n(-c2ccccc2)c1N,266.018370,1,1,1,1,1,1.701141,0.024041,1.725182,0.001041
